Documentation for `h2syn` and `hinfsyn` are currently a little sparse in the python `control` module. 
Looking at the comments for the underlying slicot function provides some help.

* `h2syn`
 * Octave documentation: https://octave.sourceforge.io/control/function/h2syn.html
 * slicot function: http://slicot.org/objects/software/shared/doc/SB10HD.html
 
* `hinfsyn`
 * octave documentation: https://octave.sourceforge.io/control/function/hinfsyn.html
 * slicot function: http://slicot.org/objects/software/shared/doc/SB10AD.html
 
The Octave documentation for both functions calls the same slicot function as control (and some additional ones).

A simple observable controlable 1st order system.

In [1]:
import control
import numpy as np

In [2]:
A = np.array([-1])
B = np.array([[1]])
C = np.array([[1]])
D = np.array([[0]])
P = control.ss(A,B,C,D)
P

A = [[-1]]

B = [[1]]

C = [[1]]

D = [[0]]

## $H_2$ synthesis

Modify the plant so $H_2$ controller synthesis generates the LQG controller, from 9.3.3 pg 389 of [1].

[1] Skogestad and Postlethwaite 1996, _Multivariable Feedback Control_.

Assume unitary weight functions ($Q,R$) and noise:

In [3]:
B1 = np.array([[1, 0]])
C1 = np.array([[1],[0]])
D11 = np.zeros((2,2))
D12 = np.array([[0],[1]])
D21 = np.array([[0,1]])

In [4]:
P_h = control.ss(A, np.hstack([B1,B]), np.vstack([C1,C]), 
                 np.block([[D11,D12], [D21,D]]))
P_h

A = [[-1]]

B = [[1 0 1]]

C = [[1]
 [0]
 [1]]

D = [[0. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]

In [5]:
K_H2 = control.h2syn(P_h,1,1)
K_H2

A = [[-1.82842712]]

B = [[0.41421356]]

C = [[-0.41421356]]

D = [[0.]]

Compare with `lqr`.

In [6]:
K_statefeedback,_,_ = control.lqr(P.A, P.B,1,1)
K_statefeedback

array([[0.41421356]])

Sign difference between `K_statefeedback` from `lqr` and `K_h2.C` as `lqr` assumes negative feedback.

## $H_\infty$ synthesis

In [7]:
K_Hoo,CL,gam_nom,rcond_nom = control.hinfsyn(P_h,1,1)

Controller:

In [8]:
K_Hoo

A = [[-23451938.14311943]]

B = [[0.7320508]]

C = [[-23451937.77709401]]

D = [[0.]]

Closed loop system $H_\infty$:

In [9]:
CL

A = [[-1.00000000e+00 -2.34519378e+07]
 [ 7.32050799e-01 -2.34519381e+07]]

B = [[1.        0.       ]
 [0.        0.7320508]]

C = [[ 1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.34519378e+07]]

D = [[0. 0.]
 [0. 0.]]

$H_\infty$ norm of the closed loop system

In [10]:
gam_nom

0.7320508099833613

### Changing Control Authority

For fun, what happens when the control authority is reduced?

In [11]:
C = np.array([[.001]])

The system remains controlable ($C$ still has rank 1).

In [12]:
P_h = control.ss(A, np.hstack([B1,B]), np.vstack([C1,C]), 
                 np.block([[D11,D12], [D21,D]]))

In [13]:
K_Hoo,CL,gam_red,rcond_red = control.hinfsyn(P_h,1,1)

The returned optimal $H_\infty$ controller is barely contractive.

In [14]:
gam_red

0.9999995048955715

What happens when the control authority increases?

In [15]:
C = np.array([[100]])
P_h = control.ss(A, np.hstack([B1,B]), np.vstack([C1,C]), 
                 np.block([[D11,D12], [D21,D]]))
K_Hoo,CL,gam_inc,rcond_inc = control.hinfsyn(P_h,1,1)

The returned controller has improved $H_\infty$ gain.

In [16]:
gam_inc

0.7071067870690273